# Initialize notebook

In [2]:
!pip install pandas geopandas numpy IPython -q

import pandas as pd
import geopandas as gpd
import numpy as np
from IPython.display import display

In [3]:
import os

project_root = os.getcwd()
os.listdir(project_root)

['.config', 'sample_data']

In [4]:
# from google.colab import drive
# drive.mount ('/content/gdrive/', force_remount=True)

# Data preprocessing
* Group and sum the election results by Municipio
* Dissolve the census sectors by Municipio and filter out non-relevant columns (higher than state-level aggregation and urban sprawl flag)

# Processing electoral results
* Download the zipped data from TSE, extract in memory and load into a pandas DataFrame
* Filter the relevant columns and group the votes by municipio

In [5]:
import requests
import zipfile
import os
import pandas as pd
from io import BytesIO

# Define the URL for the dataset
UF = 'PE'
tse_cdn_url = f'https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao_2022_{UF}.zip'

# Download the zip file
response = requests.get(tse_cdn_url)
if response.status_code == 200:
    # Extract the zip file in memory
    with zipfile.ZipFile(BytesIO(response.content)) as zfile:
        # Find the .csv file within the extracted directory
        for file_name in zfile.namelist():
            if file_name.endswith('.csv'):
                # Read the CSV file directly from the zip archive
                with zfile.open(file_name) as csvfile:
                    votacao_UF = pd.read_csv(csvfile, delimiter=";", encoding="latin 1")

                # Print shape and column names
                print(votacao_UF.shape)
                print(votacao_UF.columns)

(2763316, 26)
Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'CD_CARGO', 'DS_CARGO',
       'NR_VOTAVEL', 'NM_VOTAVEL', 'QT_VOTOS', 'NR_LOCAL_VOTACAO',
       'SQ_CANDIDATO', 'NM_LOCAL_VOTACAO', 'DS_LOCAL_VOTACAO_ENDERECO'],
      dtype='object')


In [6]:
cols = ['NR_TURNO', 'CD_MUNICIPIO', 'NM_MUNICIPIO', 'CD_CARGO', 'NR_VOTAVEL', 'SQ_CANDIDATO', 'QT_VOTOS']
cols_ = ['NR_TURNO', 'CD_MUNICIPIO', 'NM_MUNICIPIO', 'CD_CARGO', 'NR_VOTAVEL', 'SQ_CANDIDATO']

output_dir = f'intermediate'
# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

voto_UF_abrev = votacao_UF[cols]
voto_UF_abrev = voto_UF_abrev.groupby(cols_, as_index=False)['QT_VOTOS'].sum()
voto_UF_abrev.columns = ['NR_TURNO', 'CD_MUN', 'NM_MUN', 'CD_CARGO', 'NR_VOTAVEL', 'SQ_CANDIDATO', 'QT_VOTOS']
voto_UF_abrev.to_csv(f"{project_root}/intermediate/votacao_mun_2022_{UF}_abrev.csv", index=False)

print(voto_UF_abrev.shape)
print(voto_UF_abrev.columns)

(83422, 7)
Index(['NR_TURNO', 'CD_MUN', 'NM_MUN', 'CD_CARGO', 'NR_VOTAVEL',
       'SQ_CANDIDATO', 'QT_VOTOS'],
      dtype='object')


# Processing geographical geometries
* Download the zipped data from IBGE, extract in memory and load into a geopandas GeoDataFrame
* Filter the relevant columns and group the shapes by municipio

In [7]:
ibge_cdn_url = f"https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios_preliminares/malha_com_atributos/setores/json/UF/{UF}/{UF}_Malha_Preliminar_2022.zip"

# Download the zip file
response = requests.get(ibge_cdn_url)
if response.status_code == 200:
    # Extract the zip file in memory
    with zipfile.ZipFile(BytesIO(response.content)) as zfile:
        # Find the (geo) .json file within the extracted directory
        for file_name in zfile.namelist():
            if file_name.endswith('.json'):
                # Read the json file directly from the zip archive
                with zfile.open(file_name) as jsonfile:
                   malha_UF = gpd.read_file(jsonfile)

                # Print shape and column names
                print(malha_UF.shape)
                print(malha_UF.columns)

(17574, 30)
Index(['CD_SETOR', 'AREA_KM2', 'CD_REGIAO', 'NM_REGIAO', 'CD_UF', 'NM_UF',
       'CD_MUN', 'NM_MUN', 'CD_DIST', 'NM_DIST', 'CD_SUBDIST', 'NM_SUBDIST',
       'CD_MICRO', 'NM_MICRO', 'CD_MESO', 'NM_MESO', 'CD_RGI', 'NM_RGI',
       'CD_RGINT', 'NM_RGINT', 'CD_CONCURB', 'NM_CONCURB', 'v0001', 'v0002',
       'v0003', 'v0004', 'v0005', 'v0006', 'v0007', 'geometry'],
      dtype='object')


In [8]:
cols = ['CD_MUN', 'NM_MUN', 'CD_MICRO', 'NM_MICRO', 'CD_MESO',
        'NM_MESO', 'CD_RGI', 'NM_RGI', 'CD_RGINT', 'NM_RGINT', 'v0001', 'geometry']
cols_ = ['NM_MUN', 'CD_MICRO', 'NM_MICRO', 'CD_MESO',
        'NM_MESO', 'CD_RGI', 'NM_RGI', 'CD_RGINT', 'NM_RGINT']

agg_dict = {
    item: 'first' for item in cols_
}

agg_dict['v0001'] = 'sum'

malha_UF_abrev = malha_UF[cols].dissolve(by='CD_MUN', aggfunc=agg_dict, as_index=False)
malha_UF_abrev[cols]

malha_UF_abrev.to_file(f'{project_root}/intermediate/malha_{UF}_abrev.geojson', driver='GeoJSON')

print(malha_UF_abrev.shape)
print(malha_UF_abrev.columns)

(185, 12)
Index(['CD_MUN', 'geometry', 'NM_MUN', 'CD_MICRO', 'NM_MICRO', 'CD_MESO',
       'NM_MESO', 'CD_RGI', 'NM_RGI', 'CD_RGINT', 'NM_RGINT', 'v0001'],
      dtype='object')


# Processing state candidate data
* Download the zipped data from TSE, extract in memory and load into a geopandas GeoDataFrame
* Filter the relevant columns and select only candidates from the selected state

In [9]:
# Define the URL for the dataset
tse_cdn_url_cand = f'https://cdn.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand_2022.zip'

df_list = []

# Download the zip file
response = requests.get(tse_cdn_url_cand)
if response.status_code == 200:
    # Extract the zip file in memory
    with zipfile.ZipFile(BytesIO(response.content)) as zfile:
        # Find the .csv file within the extracted directory
        for file_name in zfile.namelist():
            if file_name.endswith('BRASIL.csv'):
                # Read the CSV file directly from the zip archive
                with zfile.open(file_name) as csvfile:
                    cand_BR = pd.read_csv(csvfile, delimiter=";", encoding="latin 1")

                # Print shape and column names
                print(cand_BR.shape)
                print(cand_BR.columns)

(29314, 50)
Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA_ELEICAO', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_CARGO',
       'DS_CARGO', 'SQ_CANDIDATO', 'NR_CANDIDATO', 'NM_CANDIDATO',
       'NM_URNA_CANDIDATO', 'NM_SOCIAL_CANDIDATO', 'NR_CPF_CANDIDATO',
       'DS_EMAIL', 'CD_SITUACAO_CANDIDATURA', 'DS_SITUACAO_CANDIDATURA',
       'TP_AGREMIACAO', 'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO',
       'NR_FEDERACAO', 'NM_FEDERACAO', 'SG_FEDERACAO',
       'DS_COMPOSICAO_FEDERACAO', 'SQ_COLIGACAO', 'NM_COLIGACAO',
       'DS_COMPOSICAO_COLIGACAO', 'SG_UF_NASCIMENTO', 'DT_NASCIMENTO',
       'NR_TITULO_ELEITORAL_CANDIDATO', 'CD_GENERO', 'DS_GENERO',
       'CD_GRAU_INSTRUCAO', 'DS_GRAU_INSTRUCAO', 'CD_ESTADO_CIVIL',
       'DS_ESTADO_CIVIL', 'CD_COR_RACA', 'DS_COR_RACA', 'CD_OCUPACAO',
       'DS_OCUPACAO', 'CD_SIT_TOT_TURNO', 'DS_SIT_TOT_TURNO'],
      dtype='object')


In [10]:
# situacao_cand_key = cand_PE[['CD_SITUACAO_CANDIDATURA', 'DS_SITUACAO_CANDIDATURA']].drop_duplicates()
# situacao_eleicao_key = cand_PE[['CD_SIT_TOT_TURNO', 'DS_SIT_TOT_TURNO']].drop_duplicates()

# TODO: collect gender, education, racial data in order to allow for filtering candidates by those traits
cols = ['NR_TURNO', 'CD_CARGO', 'SQ_CANDIDATO', 'NM_URNA_CANDIDATO', 'NM_CANDIDATO', 'NR_CANDIDATO', 'NR_PARTIDO', 'SG_PARTIDO', 'CD_SITUACAO_CANDIDATURA', 'CD_SIT_TOT_TURNO']

cand_UF = cand_BR[cand_BR['SG_UF'] == UF]
cand_UF_abrev = cand_UF[cols].copy()
cand_UF_abrev.to_csv(f"{project_root}/intermediate/cand_2022_{UF}_abrev.csv", index=False)

print(cand_UF_abrev.shape)
print(cand_UF_abrev.columns)

(1142, 10)
Index(['NR_TURNO', 'CD_CARGO', 'SQ_CANDIDATO', 'NM_URNA_CANDIDATO',
       'NM_CANDIDATO', 'NR_CANDIDATO', 'NR_PARTIDO', 'SG_PARTIDO',
       'CD_SITUACAO_CANDIDATURA', 'CD_SIT_TOT_TURNO'],
      dtype='object')


# Loading candidate official photos
* Download the zipped data from TSE
* Extract in memory, standardize the extension (all to .jpg) and save to the render directory

In [11]:
# Define the URL for the dataset
tse_cdn_url_photos = f'https://cdn.tse.jus.br/estatistica/sead/eleicoes/eleicoes2022/fotos/foto_cand2022_{UF}_div.zip'

output_dir = f'render/fotos{UF}2022'
# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Download the zip file
response = requests.get(tse_cdn_url_photos)
if response.status_code == 200:
    # Extract the zip file in memory
    with zipfile.ZipFile(BytesIO(response.content)) as zfile:
        # Loop through all the files in the zip archive
        for file_name in zfile.namelist():
            # Only consider .jpeg and .jpg files
            if file_name.lower().endswith(('.jpeg', '.jpg')):
                # Remove the first three characters and "_div"
                base_name = os.path.basename(file_name)
                numeric_part = base_name[3:].replace('_div', '')

                # Remove the existing file extension using string slicing
                numeric_part_without_ext = numeric_part.rsplit('.', 1)[0]

                # Standardize the extension to .jpg
                new_file_name = numeric_part_without_ext + '.jpg'

                # Save the file with the new extension in the output directory
                with zfile.open(file_name) as source_file:
                    output_path = os.path.join(output_dir, os.path.basename(new_file_name))
                    with open(output_path, 'wb') as dest_file:
                        dest_file.write(source_file.read())
        print(f'Downloaded and extracted photos to {output_dir}')

Downloaded and extracted photos to render/fotosPE2022


# Load preprocessed data


In [12]:
voto_UF_abrev = pd.read_csv(f"{project_root}/intermediate/votacao_mun_2022_{UF}_abrev.csv")
malha_UF_abrev = gpd.read_file(f'{project_root}/intermediate/malha_{UF}_abrev.geojson')
cand_UF_abrev = pd.read_csv(f"{project_root}/intermediate/cand_2022_{UF}_abrev.csv")

# Fix and fill data
* TSE (electoral court) and IBGE (geography and statistics institute) both use codes for Municipios, but the systems are no compatible; fuzzy matching by municipio name allows us to assign the IBGE codes to the municipios in the electoral results
* Compute the total votes and total valid votes for each municipio to calculate metrics for candidates
* Separate the data of interest: nominal votes and legend (party) votes
* Filter municipio geographic data to only contain IBGE code, name and shapes

In [13]:
try:
  from rapidfuzz import process, fuzz
except:
  !pip install rapidfuzz -q
  from rapidfuzz import process, fuzz

# municipios ground truth
municipios_gt = malha_UF_abrev[['CD_MUN','NM_MUN']].drop_duplicates()
municipios_gt['NM_MUN'] = municipios_gt['NM_MUN'].str.lower().str.strip()

match_dict = {}

def fuzzy_match(mun_name, mun_df, min_score=80):
    if mun_name in match_dict:
        return match_dict[mun_name]
    best_match = process.extractOne(mun_name.lower().strip(), mun_df['NM_MUN'], scorer=fuzz.ratio, score_cutoff=min_score)
    if best_match:
        matched_row = mun_df[mun_df['NM_MUN'] == best_match[0]]
        code = matched_row['CD_MUN'].values[0]
        match_dict[mun_name] = code
        return code
    return None

voto_UF_corrected = voto_UF_abrev.copy()
display(voto_UF_corrected.head())
voto_UF_corrected['CD_MUN'] = voto_UF_corrected['NM_MUN'].apply(lambda x: fuzzy_match(x, municipios_gt))
voto_UF_corrected.drop(columns=['NM_MUN'], inplace=True)
voto_UF_corrected = voto_UF_corrected[['CD_MUN', 'NR_TURNO', 'CD_CARGO', 'NR_VOTAVEL', 'SQ_CANDIDATO', 'QT_VOTOS']]

display(voto_UF_corrected.head())

voto_UF_corrected.to_csv(f"{project_root}/intermediate/votacao_mun_2022_{UF}_corrected.csv", index=False)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.7 MB/s eta 0:00:00


,NR_TURNO,CD_MUN,NM_MUN,CD_CARGO,NR_VOTAVEL,SQ_CANDIDATO,QT_VOTOS
0,1,23000,QUIXABA,3,14,170001618273,7
1,1,23000,QUIXABA,3,22,170001602587,213
2,1,23000,QUIXABA,3,40,170001618087,1011
3,1,23000,QUIXABA,3,44,170001604111,2044
4,1,23000,QUIXABA,3,45,170001604087,56


,CD_MUN,NR_TURNO,CD_CARGO,NR_VOTAVEL,SQ_CANDIDATO,QT_VOTOS
0,2611533,1,3,14,170001618273,7
1,2611533,1,3,22,170001602587,213
2,2611533,1,3,40,170001618087,1011
3,2611533,1,3,44,170001604111,2044
4,2611533,1,3,45,170001604087,56


In [44]:
voto_mun = voto_UF_corrected.copy()
voto_mun_valido = voto_UF_corrected[voto_UF_corrected['SQ_CANDIDATO'] != -1]
voto_mun_nominal = voto_UF_corrected[(voto_UF_corrected['SQ_CANDIDATO'] != -1) | (voto_UF_corrected['SQ_CANDIDATO'] == -3)]
voto_mun_legenda = voto_UF_corrected[voto_UF_corrected['SQ_CANDIDATO'] == -3]

# Step 1: Copy the data
voto_mun_partido = voto_mun_valido.copy()

# Step 2: Modify 'NR_VOTAVEL' to only keep the first two characters
voto_mun_partido['NR_VOTAVEL'] = voto_mun_partido['NR_VOTAVEL'].astype(str).str[:2]

# Step 3: Group by the modified 'NR_VOTAVEL' and other columns
cols_ = ['NR_TURNO', 'CD_MUN', 'CD_CARGO', 'NR_VOTAVEL']
voto_mun_partido = voto_mun_partido.groupby(cols_, as_index=False).agg({'QT_VOTOS': 'sum'}).sort_values(by=['NR_TURNO', 'CD_MUN', 'CD_CARGO', 'NR_VOTAVEL'])

# Step 4: Optionally drop 'SQ_CANDIDATO' (if no longer needed)
voto_mun_partido.drop(columns=['SQ_CANDIDATO'], inplace=True, errors='ignore')

# Step 5: Display or return the result
display(voto_mun_partido[(voto_mun_partido['CD_CARGO'] == 6) & ((voto_mun_partido['NR_VOTAVEL'] == '40'))].head(50))

voto_mun_total = voto_mun.groupby(['CD_CARGO', 'NR_TURNO', 'CD_MUN'], as_index=False).agg({'QT_VOTOS': 'sum'})
voto_mun_valido_total = voto_mun_valido.groupby(['CD_CARGO', 'NR_TURNO', 'CD_MUN'], as_index=False).agg({'QT_VOTOS': 'sum'})

,NR_TURNO,CD_MUN,CD_CARGO,NR_VOTAVEL,QT_VOTOS
40,1,2600054,6,40,7384
119,1,2600104,6,40,8115
190,1,2600203,6,40,2688
265,1,2600302,6,40,5406
339,1,2600401,6,40,365
417,1,2600500,6,40,432
490,1,2600609,6,40,1846
564,1,2600708,6,40,3483
637,1,2600807,6,40,5974
712,1,2600906,6,40,3704


In [28]:
malha_UF_mun = malha_UF_abrev[['CD_MUN', 'NM_MUN', 'geometry']].copy()
malha_UF_mun.head()

,CD_MUN,NM_MUN,geometry
0,2600054,Abreu e Lima,"POLYGON ((-34.92115 -7.92888, -34.92178 -7.929..."
1,2600104,Afogados da Ingazeira,"POLYGON ((-37.60467 -7.86002, -37.60408 -7.861..."
2,2600203,Afrânio,"POLYGON ((-41.04232 -8.78402, -41.04393 -8.784..."
3,2600302,Agrestina,"POLYGON ((-35.94532 -8.53637, -35.9476 -8.5351..."
4,2600401,Água Preta,"POLYGON ((-35.55924 -8.83185, -35.55924 -8.831..."


# Saving to file the components necessary for rendering

In [45]:
cand_UF_abrev.to_csv(f"{project_root}/render/cand_abrev_{UF}_2022.csv", index=False)

voto_mun.to_csv(f"{project_root}/render/voto_mun_{UF}_2022.csv", index=False)
voto_mun_valido.to_csv(f"{project_root}/render/voto_mun_valido_{UF}_2022.csv", index=False)
voto_mun_partido.to_csv(f"{project_root}/render/voto_mun_partido_{UF}_2022.csv", index=False)

voto_mun_total.to_csv(f"{project_root}/render/voto_mun_{UF}_total_2022.csv", index=False)
voto_mun_valido_total.to_csv(f"{project_root}/render/voto_mun_valido_{UF}_total_2022.csv", index=False)

malha_UF_mun.to_file(f"{project_root}/render/malha_{UF}_mun.geojson", driver='GeoJSON')

In [30]:
!zip -r render_data.zip render/

  adding: render/ (stored 0%)
  adding: render/voto_mun_valido_PE_total_2022.csv (deflated 69%)
  adding: render/voto_mun_PE_2022.csv (deflated 83%)
  adding: render/cand_abrev_PE_2022.csv (deflated 70%)
  adding: render/voto_mun_PE_total_2022.csv (deflated 76%)
  adding: render/fotosPE2022/ (stored 0%)
  adding: render/fotosPE2022/170001618417.jpg (deflated 0%)
  adding: render/fotosPE2022/170001619700.jpg (deflated 0%)
  adding: render/fotosPE2022/170001723379.jpg (deflated 2%)
  adding: render/fotosPE2022/170001609283.jpg (deflated 0%)
  adding: render/fotosPE2022/170001609301.jpg (deflated 0%)
  adding: render/fotosPE2022/170001608962.jpg (deflated 3%)
  adding: render/fotosPE2022/170001613271.jpg (deflated 0%)
  adding: render/fotosPE2022/170001609030.jpg (deflated 1%)
  adding: render/fotosPE2022/170001619986.jpg (deflated 0%)
  adding: render/fotosPE2022/170001604084.jpg (deflated 0%)
  adding: render/fotosPE2022/170001610249.jpg (deflated 1%)
  adding: render/fotosPE2022/170001